# Mike Babb
# babbm@uw.edu
# Find anagrams
## Part 2: Generate and store the anagrams v2.0

In [ ]:
# standard libraries - installed by default
import collections
import datetime
import pickle
import sqlite3
import string
import os
import timeit

In [ ]:
# external libraries - not installed by default
import numpy as np
import pandas as pd

In [ ]:
from part_00_file_db_utils import *
from part_00_process_functions import *

### set input and output paths

In [ ]:
# base file path
base_file_path = '/project/finding_anagrams'

In [ ]:
# input path
in_file_path = 'data'
in_file_path = os.path.join(base_file_path, in_file_path)

In [ ]:
# output db path and name
db_path = 'db'
db_path = os.path.join(base_file_path, db_path)

In [ ]:
if os.path.exists(db_path):
    pass
else:
    os.makedirs(db_path)

In [ ]:
db_name = 'words.db'

### process control flags

In [ ]:
# Use numpy to perform matrix opertions and determine from/to and exact anagram relationships
# Option 1: Full matrix
# Option 2: Word-length
# Option 3: First letter
# Option 4: Single-least common letter
# Option 5: n least common letters
# Option 6: word-length and n least common letters

matrix_extraction_option = 6

# max number of letters to slice to use for the generation of sub-matrices for
# options 5 and 6. More letters means more sub-matrices
# 3 seems to be the sweet spot
n_subset_letters = 3

# set write_data to True to store the generated list of anagrams
write_data = False

# set to None to include all letters
# test with a subset of letters by setting the letter_subset_list to ['q', 'x'] or 
# a different set of letters
letter_subset_list = ['x']
# letter_subset_list = None

# generate a sample dataset of ten words that start with each letter

# testo = wg_df.groupby(['first_letter']).sample(n = 10, random_state = 123)


In [ ]:
# start a timer to record the entire operation
total_time_start = datetime.datetime.now()

### load input data

In [ ]:
word_df, wg_df, letter_dict, char_matrix, word_group_id_list, word_id_list, wchar_matrix = load_input_data(db_path = db_path, db_name = db_name, in_file_path = in_file_path)

In [ ]:
# Split the char_matrix into N sub matrices
# See split_matrix() for a more elaborate description. 
# This function does a lot of things. Effectively, it computes and stores values in the wg_df, and splits the matrix into various components.

In [ ]:
wg_df, n_char_matrix_dict, single_letter_matrix_dict, letter_selector_matrix_dict, nc_ls_matrix_dict= split_matrix(
    letter_dict = letter_dict,
    word_group_id_list = word_group_id_list,
        wg_df = wg_df,
    wchar_matrix = wchar_matrix, 
    n_subset_letters = n_subset_letters,
)

# demonstrate the different matrix extraction options with the word 'achiever'

In [ ]:
demo_word = 'achiever'

wg_id = word_df.loc[word_df['lcase'] == demo_word, 'word_group_id'].iloc[0]

demo_wg_df = wg_df.loc[wg_df['word_group_id'] == wg_id, : ]

# option 1 - Full matrix
# No additional data needed

# option 2 -  Number of characters
n_char = demo_wg_df['n_chars'].iloc[0]

# option 3 - First letter
first_letter = demo_wg_df['first_letter'].iloc[0]

# option 4 - Least common letter
least_common_letter = demo_wg_df['letter_selector'].iloc[0][0]

# option 5 - Multiple least common letters
letter_selector = demo_wg_df['letter_selector'].iloc[0]

# option 6 - Number of characters and multiple least common letters
nc_ls_tuple = demo_wg_df['nc_ls_tuple'].iloc[0]


## Select on the full matrix: option 1

In [ ]:
# demo the full matrix selection
output = get_values_full_matrix(wg_id = wg_id, 
                    wchar_matrix = wchar_matrix,
                   word_group_id_list = word_group_id_list)

# this is an array of from words to the word 'achiever'
format_demo_output(demo_word = demo_word,
                   word_df = word_df,
                   demo_output = output)

## Select on the matrices split by word-length: option 2

In [ ]:
# demo the n char selection
output = get_values_n_char(wg_id = wg_id,
                      n_char = n_char,
                      n_char_matrix_dict = n_char_matrix_dict)

# this is an array of from words to the word 'achiever'
format_demo_output(demo_word = demo_word,
                   word_df = word_df,
                   demo_output = output)

## Select on the matrices split by the first letter: option 3

In [ ]:
# demo the first letter selection
output = get_values_single_letter(wg_id = wg_id, single_letter = least_common_letter,
                                                           single_letter_matrix_dict = single_letter_matrix_dict)         

# this is an array of from words to the word 'achiever'
format_demo_output(demo_word = demo_word,
                   word_df = word_df,
                   demo_output = output)

## Select on the matrices split by the single least common letter: option 4

In [ ]:
# demo the first letter selection
output = get_values_single_letter(wg_id = wg_id, single_letter = least_common_letter,
                                                           single_letter_matrix_dict = single_letter_matrix_dict)         

# this is an array of from words to the word 'achiever'
format_demo_output(demo_word = demo_word,
                   word_df = word_df,
                   demo_output = output)

## Select on the matrices split by the letter selector: option 5

In [ ]:
# demo with the letter selector
output = get_values_letter_selector(wg_id = wg_id,
                      letter_selector = letter_selector,
                      letter_selector_matrix_dict = letter_selector_matrix_dict)

# this is an array of from words to the word 'achiever'
format_demo_output(demo_word = demo_word,
                   word_df = word_df,
                   demo_output = output)

## Select on the matrices split by word-length and the letter selector: option 6

In [ ]:
# demo with the n_char letter selector
output = get_values_n_char_letter_selector(wg_id = wg_id,
                           nc_ls_tuple = nc_ls_tuple,                           
                           nc_ls_matrix_dict=nc_ls_matrix_dict)

# this is an array of from words to the word 'achiever'
format_demo_output(demo_word = demo_word,
                   word_df = word_df,
                   demo_output = output)

In [ ]:
# we've tested with one word, let's time many evaluations to get a sense of how quickly 
# the different matrix extraction options work
# use the timeit() function to evaluate how long, on average, a single matrix operation
# takes to complete
# we do this by writing python code encapsulated in quotes which is then sent to the function
# we can store the quoted code in dictionary and then enumerate. 

n_trials = 10

code_snippet_dict = {
    'Matrix Selection Option 1: Selecting by full matrix':
"""get_values_full_matrix(wg_id = wg_id, wchar_matrix = wchar_matrix, word_group_id_list = word_group_id_list)""",
    'Matrix Selection Option 2: Selecting by word length':
"""get_values_n_char(wg_id = wg_id, n_char = n_char, n_char_matrix_dict = n_char_matrix_dict)""",
    'Matrix Selection Option 3: Selecting by first letter':
"""get_values_single_letter(wg_id = wg_id, single_letter = first_letter, single_letter_matrix_dict = single_letter_matrix_dict)""",    
    'Matrix Selection Option 4: Selecting by single least common letter':
"""get_values_single_letter(wg_id = wg_id, single_letter = least_common_letter, single_letter_matrix_dict = single_letter_matrix_dict)""",
    'Matrix Selection Option 5: Selecting by letter selector':
"""get_values_letter_selector(wg_id = wg_id, letter_selector = letter_selector, letter_selector_matrix_dict = letter_selector_matrix_dict)""",
    'Matrix Selection Option 6: Selecting by word length and letter selector':
"""get_values_n_char_letter_selector(wg_id = wg_id, nc_ls_tuple = nc_ls_tuple, nc_ls_matrix_dict=nc_ls_matrix_dict)"""
}

timing_list = []
for csd, cs in code_snippet_dict.items():    

    # here we time the code execution
    total_time = timeit.timeit(cs, number=n_trials, globals=globals())
    timing_list.append([csd, total_time])

    # total time    
    total_time_formatted = '{:,}'.format(round(total_time, 4))    

    # average time
    avg_time = total_time / n_trials
    avg_time_formatted = '{:,}'.format(round(avg_time, 6)) 
        
    print(csd)
    # average number of seconds per trial
    print('Total time:', total_time_formatted, 'seconds. Average time:', avg_time_formatted, 'seconds.')

    # add a line break to make it easier to read
    print()
    
    

In [ ]:
# the combination of the word length and letter selector is the fastest

In [ ]:
# these numbers are orders of magnitude different and therefore hard to interpret.
# let's rescale.

In [ ]:
from itertools import product

In [ ]:
my_string = 'Matrix Selection Option 1: This is the stuff'

In [ ]:
my_string[17:25]

In [ ]:
expanded_timing_list = []
for me_source, me_target in product(timing_list, repeat=2):    
    # let's unpack this
    me_source_option, me_source_timing = me_source
    me_target_option, me_target_timing = me_target

    me_source_option = me_source_option[17:25]
    me_target_option = me_target_option[17:25]
    me_source_target_timing_ratio = me_source_timing / me_target_timing
    
    # add to the list
    expanded_timing_list.append([me_source_option, me_target_option, me_source_timing, me_target_timing, me_source_target_timing_ratio])
    
    

    

    

In [ ]:
col_names = ['me_source', 'me_target', 'me_source_timing', 'me_target_timing', 'me_timing_ratio']

In [ ]:
timing_df = pd.DataFrame(data = expanded_timing_list, columns = col_names)

In [ ]:
timing_df.head()

In [ ]:
# that's cool and all, but let's see if we can visualize the whole thing? 

In [ ]:
# use the pivot option

In [ ]:
timing_table = timing_df.pivot_table(index = ['me_source'], columns = ['me_target'],
                           values =['me_timing_ratio'])

In [ ]:
timing_table